In [1]:
import os
import tempfile

from langchain.document_loaders import PyPDFLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from dotenv import load_dotenv
import pinecone # noqa
import tiktoken
from PyPDF2 import PdfReader

C:\Users\torre\Documents\Dev\chat-with-docs\venv\lib\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
load_dotenv()

True

Lectura de pdf y conversion a un solo texto

In [7]:
docs = []
temp_dir = tempfile.TemporaryDirectory()
folder = 'data'
texto = ""

for file in os.listdir(folder):
    path_file = os.path.join(folder, file)
    reader = PdfReader(path_file)
    
    metadata = {
        "titulo": file.replace("_", " ").replace(".pdf", ""),
        "tipo": "Propuesta de Constitución" if "propuesta" in file.lower() else "Constitución Vigente",
    }
    
    texto = "--------------------------\n"
    texto += f"INICIO DEL DOCUMENTO\nTITULO: {metadata['titulo']}\nTIPO: {metadata['tipo']}\n"
    
    for pagina in reader.pages:
        texto += pagina.extract_text() + '\n'
        
    texto += "FIN DEL DOCUMENTO\n"
    texto += "--------------------------\n"
    
    with open(f'text/{file.replace(".pdf", ".txt")}', 'w', encoding='utf-8') as archivo_txt:
        archivo_txt.write(texto)
        
# Crear loaders
folder = 'text'

for file in os.listdir(folder):
    print(file)
    
    file_path = os.path.join(folder, file)
    loader = TextLoader(file_path, encoding='utf-8')
    docs.extend(loader.load())

CONSTITUCIÓN_POLÍTICA_DE_LA_REPÚBLICA.txt
porpuesta_de_nueva_constitucion_2022.txt
porpuesta_de_nueva_constitucion_2023.txt


Lectura de pdf y creación de loader

In [6]:
docs = []
temp_dir = tempfile.TemporaryDirectory()
folder = 'data'

for file in os.listdir(folder):
    print(file)
    
    file_path = os.path.join(folder, file)
    loader = PyPDFLoader(file_path=file_path)

    docs.extend(loader.load_and_split())

CONSTITUCIÓN_POLÍTICA_DE_LA_REPÚBLICA.pdf
porpuesta_de_nueva_constitucion_2022.pdf
porpuesta_de_nueva_constitucion_2023.pdf


In [8]:
len(docs)

3

In [9]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
splits = text_splitter.split_documents(docs)

In [10]:
len(splits)

2661

In [11]:
def consultar_costo_embedding(p):
    enc = tiktoken.encoding_for_model('text-embedding-ada-002')
    total_tokens = sum([len(enc.encode(page.page_content)) for page in p])
    print(f"Total tokens {total_tokens}")
    print(f"Costo embedding {total_tokens / 1000 * 0.0001:.6f}")

In [12]:
consultar_costo_embedding(splits)

Total tokens 330629
Costo embedding 0.033063


In [13]:
api_key = os.getenv('OPENAI_API_KEY')
embeddings = OpenAIEmbeddings(openai_api_key=api_key)

In [14]:
api_pinecone = os.getenv('PINECONE_API_KEY')
env_pinecone = os.getenv('PINECONE_ENV')

pinecone.init(
    api_key=api_pinecone,
    environment=env_pinecone
)
index_name = "constitution-idx"


In [15]:
if index_name not in pinecone.list_indexes():
    pinecone.create_index(name=index_name, metric="cosine", dimension=1536)

# docsearch = Pinecone.from_text([t.page_content for t in pages], embeddings, index_name=index_name)
# docsearch = Pinecone.from_documents(pages, embeddings, index_name=index_name)
docsearch = Pinecone.from_documents(splits, embeddings, index_name=index_name)

# docsearch = Pinecone.from_existing_index(index_name, embeddings)

index = pinecone.Index(index_name)
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 2661}},
 'total_vector_count': 2661}

In [23]:
docsearch.similarity_search(
    "2023",
    k=10
)

[Document(page_content='marzo de 2026. Las y los actuales integrantes del Senado terminarán sus \nmandatos el 11 de marzo de 2026 y podrán postular a las elecciones para el \nCongreso de Diputadas y Diputados y la Cámara de las Regiones que se realizará en noviembre de 2025, donde serán elegidas las diputadas y \ndiputados y representantes regionales que ejercerán sus funci ones desde el \n11 de marzo de 2026. De ser electos en los comicios celebrados en 2025 para', metadata={'source': 'text\\porpuesta_de_nueva_constitucion_2022.txt'}),
 Document(page_content='Congreso Nacional en la ciudad de Santiago, con fecha 6 de marzo de 2023, e iniciará la \nredacción de un anteproyecto de nueva Cons titución. En la misma fecha, entrará en \nfunciones el Comité Técnico de Admisibilidad. \nLa Comisión Experta deberá aprobar cada norma que formará parte del \nanteproyecto de propuesta de nueva Constituci ón por un quórum de los tres quintos de \nsus miembros en ejercicio. \nEl anteproyecto de nuev

In [ ]:
# retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 4})